In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-word2vec

cd container

chmod +x decision_trees/train
chmod +x decision_trees/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-1 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

In [ ]:
# S3 prefix
prefix = 'word2vec'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
from sagemaker.session import Session

model_data = Session().upload_data(path='model.tar.gz', key_prefix='model')
print(model_data)

In [ ]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [ ]:
# generating the image path
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-word2vec:latest'.format(account, region)

In [ ]:
# to deploy the model, you need to have the model created based on your model artifacts, 
# create an endpoint configuration and then create the endpoint based on the two.
# first let's create the model

client = boto3.client('sagemaker')
ModelName='TestCx-BYOA' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
first_model = client.create_model(
    ModelName=ModelName,
    PrimaryContainer={
        'Image': image,
        'ModelDataUrl': 's3://sagemaker-us-east-1-639634733305/output/sagemaker-word2vec-2020-01-10-20-32-37-771/output/model.tar.gz'    # note that the model.tar.gz file is a tarball of our word2vec_2.model file
        # I manually uploaded the tarbal file into this location
        },
    ExecutionRoleArn=role)

In [ ]:
# create the endpoint configuration
endpoint_config_name = 'TestCx-BYOA-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':ModelName,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

In [ ]:
# create the endpoint
endpoint_name = 'TestCx-BYOA-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])